# Lab - Univariate regression with Polynomial Features

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use("seaborn-v0_8-whitegrid")

import seaborn as sns
import pingouin as pg

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import mean_squared_error

from scipy.special import struve

from scipy.stats import norm, uniform

# The following can be used to combine pdfs into 1
#from matplotlib.backends.backend_pdf import PdfPages
#
# See https://stackoverflow.com/a/11329151
#pp = PdfPages('output/combined.pdf')
#pp.savefig(fig) # add figure using its fig handle

colours = sns.color_palette('colorblind')

## Generate Data

In [ ]:
def f1(x):
  return struve(0,10*x)

In [ ]:
X = uniform.rvs(loc=0, scale=1, size=50, random_state=42)
X.sort()

In [ ]:
true_signal = f1(X)
plt.plot(X, true_signal, 'k.-')
plt.xlabel("x")
plt.ylabel("f1")
plt.title(f"True signal f1 ({len(X)} points, x distributed according to uniform distribution)")
plt.xlim(0,1)
plt.ylim(-1,3)
#plt.savefig('output/trueSignal.pdf')
plt.show()

#### Derive Noise and display plots

In [ ]:
noise = norm.rvs(loc=0, scale=1, size=len(X), random_state=42)

In [ ]:
sns.displot(noise, kde=True)
plt.show()

In [ ]:
pg.qqplot(noise)
plt.show()

#### Add the generated noise and plot the noisy signal

In [ ]:
y = true_signal + 0.1 * noise

In [ ]:
plt.plot(X, y, "k.");
plt.xlim(0,1)
plt.ylim(-1,3)
plt.title(f"Noisy signal f1 ({len(X)} points, x distributed according to uniform distribution)")
plt.xlabel("x")
#plt.savefig('output/noisySignal.pdf')
plt.show()

## Split Data into train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=42)

In [ ]:
plt.scatter(X_train, y_train, color=colours[0], marker="+", label='train');
plt.scatter(X_test, y_test, color=colours[1], marker=".", label='test');
plt.title(f"Actual Noisy signal ({len(y_train)} train, {len(y_test)} test data)")
plt.xlabel("x")
plt.ylabel("y")
plt.xlim(0,1)
plt.ylim(-1,3)
plt.legend(frameon=True)
#plt.savefig('output/trainTest.pdf')
plt.show()

## Construct Features for a Polynomial Model

In [ ]:
def preparePolyFeatures(degree, X_train, y_train):
  poly = PolynomialFeatures(degree)
  # fit = train = determine optimal parameters using the train dataset only
  poly.fit(X_train.reshape(-1, 1))
  # transform = apply transformation to both train and test
  X_train_poly = poly.transform(X_train.reshape(-1, 1))
  X_test_poly = poly.transform(X_test.reshape(-1, 1))
  return poly, X_train_poly, X_test_poly

## For linearly-spaced points, prepare the predicted polynomial learned from the training set

In [ ]:
def preparePolyPoints(poly, model):
  # generate extra ordered points for smooth curve
  X_plot = np.linspace(0, 1, 100)
  X_plot_poly = poly.transform(X_plot.reshape(-1, 1))
  y_plot_pred = model.predict(X_plot_poly)
  return X_plot, X_plot_poly, y_plot_pred

## For the polynomial features derived from the training data, use OLS to predict the beta parameters and compute their range

In [ ]:
def olsFit(X_train_poly, y_train):
  # Fit a Linear Regression model to the polynomial features
  model = LinearRegression()
  model.fit(X_train_poly, y_train)
  betaPoly = model.coef_
  betaPolyRange = np.max(betaPoly)-np.min(betaPoly)
  return model, betaPoly, betaPolyRange

## For the polynomial features derived from the training data, use Regularised Linear Regression fit with cross-validation over the candidate lambdas, to predict the beta parameters and compute their range

In [ ]:
def regularisedLRFit(X_train_poly, y_train, lambdas):
  # Fit a regularised Linear Regression model to the polynomial features
  model = RidgeCV(alphas=lambdas, scoring = 'neg_mean_squared_error', store_cv_results=True)
  model.fit(X_train_poly, y_train)
  betaPoly = model.coef_
  cvScores = model.cv_results_
  bestLambda = model.alpha_
  betaPolyRange = np.max(betaPoly)-np.min(betaPoly)
  return model, betaPoly, betaPolyRange, cvScores, bestLambda

## Compute the mean square error between the actual and predicted targets, using the model learned from the training set

In [ ]:
def prepMSE(model, X_poly, y):
  # Compute the mean_squared_error between the actual and predicted targets
  y_pred = model.predict(X_poly)
  mseScore = mean_squared_error(y, y_pred) 
  return mseScore

## Plot training and test sets, with the predicted target from the training set

In [ ]:
def prepPlotFit(rType, X_train, y_train, X_test, y_test, X_plot, y_plot_pred, degree):
  # prepare the figure showing the fitted polynomial against the training and test data
  # global fig
  # neded so that return is not needed from this function - otherwise run into bug in notebook_exporter
  fig = plt.figure()
  plt.scatter(X_train, y_train, color=colours[0], marker="+", label='actual (train)')
  plt.scatter(X_test, y_test, color=colours[1], marker=".", label='actual (test)')
  plt.plot(X_plot, y_plot_pred, color=colours[4], label='model')
  plt.title(f"Plot (degree={degree}) {rType} polynomial fit to Actual ({len(y_train)} training, {len(y_test)} test) points")
  plt.xlabel("x")
  plt.legend(frameon=True)
  plt.ylim(-1,3)
  plt.savefig(f'output/{rType}/trainTest{degree}.pdf')
  plt.show()
  plt.close()

## Derive polynomial features for a specified degree, fit the training data, compute MSE for the training and test data, plot the results in context

In [ ]:
def fitAndPlot(X_train, y_train, Xtest, y_test, degree, rType):
  poly, X_train_poly, X_test_poly = preparePolyFeatures(degree, X_train, y_train)
  if (rType == 'OLS'):
    model, betaPoly, betaPolyRange = olsFit(X_train_poly, y_train)
    bestLambda = 0.0
  else:
    lambdas = np.logspace(-3.0, 4.0, num=16, base=10.0)
    model, betaPoly, betaPolyRange, cvLambdaScores, bestLambda = regularisedLRFit(X_train_poly, y_train, lambdas)
  # Now use K=5-fold cross-validation to estimate the MSE of the model fit, relative to the validation data (the left-out data) 
  cv_results = cross_validate(model, X_train_poly, y_train, cv=5,
               scoring=('neg_mean_squared_error'),  return_train_score=True)
  mseCV = -1*np.mean(cv_results['test_score'])
  # For comparison, we can compute the training and test MSE scores
  mseTrain = prepMSE(model, X_train_poly, y_train)
  mseTest = prepMSE(model, X_test_poly, y_test)
  X_plot, X_plot_poly, y_plot_pred = preparePolyPoints(poly, model)
  # prepare the figure showing the fitted polynomial against the training and test data
  prepPlotFit(rType, X_train, y_train, X_test, y_test, X_plot, y_plot_pred, degree)
  return mseCV, mseTrain, mseTest, betaPolyRange, betaPoly, bestLambda

## Plot the mean square error (MSE) against polynomial degree, for training and test set

In [ ]:
def plotMSEforDegrees(rType, mseDf):
  fig = plt.figure()
  plt.scatter(mseDf.index, mseDf['train'], color=colours[0], marker="+", label='train MSE')
  plt.scatter(mseDf.index, mseDf['cvMSE'], color=colours[4], marker="x", label='cv MSE')
  plt.title(f"{rType} MSE for train and cross-validation against degree")
  plt.xlabel("polynomial degree")
  plt.legend(frameon=True)
  plt.savefig(f"output/{rType}/MSEforDegrees.pdf")
  plt.show()

## Plot the range (max-min) of fitted polynomial coefficients against polynomial degree, for training and test set

In [ ]:
def plotCoeffsForDegrees(rType, mseDf):
  fig = plt.figure()
  plt.plot(mseDf.index, mseDf['betaPolyRange'], "k.-", label='range beta')
  plt.title(f"{rType} Range (max-min) beta (polynomial coefficients) against degree")
  plt.xlabel("polynomial degree")
  plt.legend(frameon=True)
  plt.savefig(f"output/{rType}/coeffsForDegrees.pdf")
  plt.show()

## Plot the regularisation parameters lambda (found using cross validation) against polynomial degree

In [ ]:
def plotLambdasForDegrees(rType, mseDf):
  fig = plt.figure()
  plt.plot(mseDf.index, mseDf['bestLambda'], "k.-", label='best lambda')
  plt.title(f"{rType} best lambda against degree")
  plt.xlabel("polynomial degree")
  plt.legend(frameon=True)
  plt.savefig(f"output/{rType}/lambdasForDegrees.pdf")
  plt.show()

## Fit, using rType procedure, a polynomial to training data, plotting the results and returning a summary

In [ ]:
def fitPlotAndSave(X_train, y_train, X_test, y_test, maxDegree, rType):
  cvMSE = []
  trainMSE = []
  testMSE = []
  #coeffs = []
  ranges = []
  bestLambdas = []
  for degree in range(0,maxDegree+1):
    mseCV, mseTrain, mseTest, betaPolyRange, betaPoly, bestLambda = fitAndPlot(X_train, y_train, X_test, y_test, degree, rType)
    
    cvMSE.append(mseCV)
    trainMSE.append(mseTrain)
    testMSE.append(mseTest)
    ranges.append(betaPolyRange)
    #coeffs.append(betaPoly)
    bestLambdas.append(bestLambda)
  
  degrees = list(range(0,maxDegree+1))
  rTypes = [rType] * len(degrees) 
  mseDf = pd.DataFrame({
    'rType': rTypes,
    'degree': degrees,
    'bestLambda': bestLambdas,
    'cvMSE': cvMSE,
    'train': trainMSE,
    'test': testMSE,
    'betaPolyRange': ranges
    })
  plotMSEforDegrees(rType, mseDf)
  plotCoeffsForDegrees(rType, mseDf)
  plotLambdasForDegrees(rType, mseDf)
  return mseDf

## "Main program": predict y using varying degrees of polynomial, with or without regularisation, with diagnostic output including plots

In [ ]:
# See https://stackoverflow.com/a/76020741 and https://stackoverflow.com/a/67548340
data = []
for rType in ('OLS', 'regularisedLR'):
  data.append(fitPlotAndSave(X_train, y_train, X_test, y_test, maxDegree=12, rType=rType))
mseDf = pd.concat(data, axis=0, ignore_index=True)
mseDf

## General comments

a) The OLS fit indicates that degree 4-6 offers good performance. Beyond this point, there are signs of overfitting.
b) Regularisation behaviour is slightly harder to interpret, but it looks that higher degree plots don't wiggle as much as they would without regularisation. So it looks like, for this case, that degree = 9 might have the lowest MSE.

Note the double use of cross-validation
1. using `RidgeCV()` to find `bestLambda` for a given polynomial degree, when regularisation is used
2. using `cross_validate()` to estimate the uncertainty in the model for a given choice of `degree`, when either OLS or regularisation is used.
